In [ ]:
!pip install sastrawi
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
#import dataset
data = pd.read_csv("/content/dataset/komentar.csv", encoding_errors= 'replace')

data.head()

,Username,Komentar
0,\n@rosasihombing3297\n,"Sangat tenang, cuek, benar2 tak ada penyesalan..."
1,\n@oldschool9449\n,Yg jelas Jessika memesan minuman tanpa comfir ...
2,\n@hendimulyadi8615\n,Masih belum lega nih sama kasus ini. Penyelesa...
3,\n@DwiCP_17\n,"Kalau aku makan bareng teman atau saudara, pes..."
4,\n@davadava7533\n,Bohong nya sudah keterlaluan si Iblis Jess!!! ...


In [ ]:
#buang kolom username
data = data.drop(columns=['Username'])
data.shape

(11, 1)

In [ ]:
#Proses cleaning dataset dari karakter-karakter yang tidak dibutuhkan
def text_prep(teks):
    teks = str(teks).lower()
    teks = re.sub('@[\w]+','',str(teks))
    teks = re.sub('[^a-zA-Z]','  ', str(teks))
    teks = re.sub(r"\b[a-zA-Z]\b", "", teks)
    teks = re.sub('\s+',' ',teks)
    return teks
data['clean'] = data['Komentar'].apply(text_prep)
data.head(10)

,Komentar,clean
0,"Sangat tenang, cuek, benar2 tak ada penyesalan...",sangat tenang cuek benar tak ada penyesalan as...
1,Yg jelas Jessika memesan minuman tanpa comfir ...,yg jelas jessika memesan minuman tanpa comfir ...
2,Masih belum lega nih sama kasus ini. Penyelesa...,masih belum lega nih sama kasus ini penyelesai...
3,"Kalau aku makan bareng teman atau saudara, pes...",kalau aku makan bareng teman atau saudara pese...
4,Bohong nya sudah keterlaluan si Iblis Jess!!! ...,bohong nya sudah keterlaluan si iblis jess lih...
5,Ga kebayang ribetnya pelayan mengatasi masalah...,ga kebayang ribetnya pelayan mengatasi masalah...
6,Gua kalo jadi pelayannya bakalan lupa sih sama...,gua kalo jadi pelayannya bakalan lupa sih sama...
7,Waduh.. telat berita 4 tahun gw min .. asw,waduh telat berita tahun gw min asw
8,"Logika aja lah, kalian klo ngopi sama temen2 k...",logika aja lah kalian klo ngopi sama temen kan...
9,Udah 2021 dan aku baru nnton berita ini diYT g...,udah dan aku baru nnton berita ini diyt gegara...


In [ ]:
#ubah data komentar keadalam bentuk token
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['tokenize'] = data['clean'].apply(word_tokenize_wrapper)

data[['clean','tokenize']]

,clean,tokenize
0,sangat tenang cuek benar tak ada penyesalan as...,"[sangat, tenang, cuek, benar, tak, ada, penyes..."
1,yg jelas jessika memesan minuman tanpa comfir ...,"[yg, jelas, jessika, memesan, minuman, tanpa, ..."
2,masih belum lega nih sama kasus ini penyelesai...,"[masih, belum, lega, nih, sama, kasus, ini, pe..."
3,kalau aku makan bareng teman atau saudara pese...,"[kalau, aku, makan, bareng, teman, atau, sauda..."
4,bohong nya sudah keterlaluan si iblis jess lih...,"[bohong, nya, sudah, keterlaluan, si, iblis, j..."
5,ga kebayang ribetnya pelayan mengatasi masalah...,"[ga, kebayang, ribetnya, pelayan, mengatasi, m..."
6,gua kalo jadi pelayannya bakalan lupa sih sama...,"[gua, kalo, jadi, pelayannya, bakalan, lupa, s..."
7,waduh telat berita tahun gw min asw,"[waduh, telat, berita, tahun, gw, min, asw]"
8,logika aja lah kalian klo ngopi sama temen kan...,"[logika, aja, lah, kalian, klo, ngopi, sama, t..."
9,udah dan aku baru nnton berita ini diyt gegara...,"[udah, dan, aku, baru, nnton, berita, ini, diy..."


In [ ]:
#normalisasi kata-kata gaul dan singkatan kedalam bentuk yang lebih baku
normalizad_word = pd.read_csv(
    "/content/kamus/singkatan.csv")
normalizad_word1 = pd.read_excel(
    "/content/kamus/normalisasi.xlsx")

normalizad_word_dict = {}
normalizad_word1_dict = {}


for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]


def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]


data['singkatan'] = data['tokenize'].apply(normalized_term)

for index, row in normalizad_word1.iterrows():
    if row[0] not in normalizad_word1_dict:
        normalizad_word1_dict[row[0]] = row[1]


def normalized_term1(document):
    return [normalizad_word1_dict[term] if term in normalizad_word1_dict else term for term in document]


data['norm'] = data['singkatan'].apply(normalized_term1)
data[['singkatan','norm']].head(20)

<ipython-input-15-5a53d119820e>:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalizad_word_dict:
<ipython-input-15-5a53d119820e>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalizad_word_dict[row[0]] = row[1]
<ipython-input-15-5a53d119820e>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalizad_word1_dict:
<ipython-input-15-5a53d119820e>:24: FutureWarning: Series.__getitem__ tre

,singkatan,norm
0,"[sangat, tenang, cuek, benar, tak, ada, penyes...","[sangat, tenang, cuek, benar, tidak, ada, peny..."
1,"[yang, jelas, jessika, memesan, minuman, tanpa...","[yang, jelas, jessika, memesan, minuman, tanpa..."
2,"[masih, belum, lega, nih, sama, kasus, ini, pe...","[masih, belum, lega, nih, sama, kasus, ini, pe..."
3,"[kalau, aku, makan, bareng, teman, atau, sauda...","[kalau, saya, makan, bareng, teman, atau, saud..."
4,"[bohong, nya, sudah, keterlaluan, si, iblis, j...","[bohong, nya, sudah, keterlaluan, sih, iblis, ..."
5,"[ga, kebayang, ribetnya, pelayan, mengatasi, m...","[tidak, terbayang, ribetnya, pelayan, mengatas..."
6,"[gua, kalau, jadi, pelayannya, bakalan, lupa, ...","[gua, kalau, jadi, pelayannya, bakalan, lupa, ..."
7,"[waduh, telat, berita, tahun, gue, min, asw]","[waduh, telat, berita, tahun, saya, admin, asw]"
8,"[logika, aja, lah, kalian, kalau, ngopi, sama,...","[logika, saja, lah, kalian, kalau, ngopi, sama..."
9,"[sudah, dan, aku, baru, nnton, berita, ini, di...","[sudah, dan, saya, baru, menonton, berita, ini..."


In [ ]:
# membuang stopword
list_stopwords = stopwords.words('indonesian')

# menambah kata kedalam kamus stopword
list_stopwords.extend(["yang", "dengan", "kan", "nya", "gimana", 'juga', 'ke', 'bikin', 'bilang', 'ingin', 'bisa', 'gimana', 'harus',
                       'nih', 'sih', 'kakak', 'terus', 'juga', 'demi', 'lah', 'luar', 'waktu', 'terus', 'pun', 'kenapa',
                       'si', 'tuh', 'untuk', 'n', 'tt', 'daripada', 'mana', 'tuh', 'siapa', 'tadi', 'pak', 'tanpa', 'atau', 'di',
                       'hehe', 'u', 'ni', 'loh', 'tu', 'bahkan', 'masi', 'masih', 'dulu', 'kali', 'nah', 'niii', 'gasi',
                       'amp', 'nih', 'seminggu', 'itupun', 'bawah', 'lu', 'gimana', 'bulan', 'yah',
                       'ka', 'tapi', 'pas', 'saat', 'satu', 'minggu', 'hari', 'orang', 'pada', 'ini', 'itu', 'hanya', 'akan', 'sini', 'sana'
                       'atau', 'dan', 'apa', 'loh', 'lo', 'kapan', 'untuk', 'akhir', 'baru', 'apalagi', 'kok', 'bagaimana', 'gimana',
                       'gapapa', 'gk', 'ga', 'senin', 'selasa', 'rabu', 'kamis', 'jumat', 'sabtu', 'minggu', 'akan', 'depan',
                       'hari', 'tahun', 'ku', 'mu', 'dia', 'anda', 'kamu', 'mereka', 'kita', 'kami', 'gitu', 'gini', 'padahal', 'siang'])
list_stopwords = set(list_stopwords)

# hapus stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]


data['stopwordremoval'] = data['norm'].apply(stopwords_removal)
data[['norm', 'stopwordremoval']].head(10)

,norm,stopwordremoval
0,"[sangat, tenang, cuek, benar, tidak, ada, peny...","[tenang, cuek, penyesalan, asli, psikopat]"
1,"[yang, jelas, jessika, memesan, minuman, tanpa...","[jessika, memesan, minuman, comfir, mirna, men..."
2,"[masih, belum, lega, nih, sama, kasus, ini, pe...","[lega, penyelesainnya, sempurna, need, sherlock]"
3,"[kalau, saya, makan, bareng, teman, atau, saud...","[makan, bareng, teman, saudara, pesan, minum, ..."
4,"[bohong, nya, sudah, keterlaluan, sih, iblis, ...","[bohong, iblis, jess, lihat, deh, taruh, bag, ..."
5,"[tidak, terbayang, ribetnya, pelayan, mengatas...","[terbayang, ribetnya, pelayan, mengatasi, terp..."
6,"[gua, kalau, jadi, pelayannya, bakalan, lupa, ...","[gua, pelayannya, lupa, kejadian, customer, de..."
7,"[waduh, telat, berita, tahun, saya, admin, asw]","[telat, berita, admin, asw]"
8,"[logika, saja, lah, kalian, kalau, ngopi, sama...","[logika, ngopi, teman, mesennya, kopi, dipesen..."
9,"[sudah, dan, saya, baru, menonton, berita, ini...","[menonton, berita, diyt, gegara, muncul, beranda]"


In [ ]:
#mengubah kata kedalam bentuk dasar dan menggabungkan list token ke bentuk komentar semula
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def text_stem(teks):
    teks = str(teks).lower()
    teks = stemmer.stem(teks)
    return teks
data['stemming'] = data['stopwordremoval'].apply(text_stem)

In [ ]:
# Fungsi untuk memeriksa apakah entri kosong atau hanya berisi spasi
def is_blank_or_whitespace(s):
    return pd.isna(s) or s.strip() == ''

# Identifikasi baris yang kosong atau hanya berisi spasi
blank_or_whitespace_rows = data['stemming'].apply(is_blank_or_whitespace)

# Hapus baris yang kosong atau hanya berisi spasi
data = data[~blank_or_whitespace_rows].reset_index(drop=True)
data.shape

(11, 7)

In [ ]:
#perbandingan sebelum dan sesudah text prerocessing pada dataset

data[['Komentar', 'stemming']]

,Komentar,stemming
0,"Sangat tenang, cuek, benar2 tak ada penyesalan...",tenang cuek sesal asli psikopat
1,Yg jelas Jessika memesan minuman tanpa comfir ...,jessika mes minum comfir mirna curiga
2,Masih belum lega nih sama kasus ini. Penyelesa...,lega penyelesainnya sempurna need sherlock
3,"Kalau aku makan bareng teman atau saudara, pes...",makan bareng teman saudara pesan minum saji se...
4,Bohong nya sudah keterlaluan si Iblis Jess!!! ...,bohong iblis jess lihat deh taruh bag kepala s...
5,Ga kebayang ribetnya pelayan mengatasi masalah...,bayang ribetnya layan atas pikir
6,Gua kalo jadi pelayannya bakalan lupa sih sama...,gua layan lupa jadi customer detail adegan rut...
7,Waduh.. telat berita 4 tahun gw min .. asw,telat berita admin asw
8,"Logika aja lah, kalian klo ngopi sama temen2 k...",logika ngopi teman mesennya kopi senin duluan
9,Udah 2021 dan aku baru nnton berita ini diYT g...,tonton berita diyt gegara muncul beranda
